https://docs.pygod.org/en/latest/generated/pygod.detector.ANOMALOUS.html


paper: https://www.ijcai.org/Proceedings/2018/0488.pdf

교수님꼐서 알려주신 사이트 https://pycaret.gitbook.io/docs/get-started/quickstart#anomaly-detection

`Summary`

- 노드당 매핑된 attribute로 이상치를 계산해낸다.
- 그래서 attribute 특징마다 나오는 이상치라고 칭하는 노드가 다른 것 같다.
- 노드 정보와 네트워크를 기반으로 rare하거나 상당히 differ한 인스턴스 집합 찾는 것을 목표로 한다.

# Import

In [1]:
import pygod
import numpy as np
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

import torch
from pygod.generator import gen_contextual_outlier, gen_structural_outlier

from pygod.utils import load_data

from pygod.metric import eval_roc_auc

from pygod.detector import ANOMALOUS

/home/csy/anaconda3/envs/temp_csy/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Tutorial?

In [2]:
data = Planetoid('./data/Cora', 'Cora', transform=T.NormalizeFeatures())[0]

`gen_contextual_outlier`의 역할: Generating contextual outliers

- 임의로 선택한 노드 중 그 노드들끼리 얼마나 떨어져 있나?

In [3]:
data, ya = gen_contextual_outlier(data, n=100, k=50)

In [4]:
ya

tensor([0, 0, 0,  ..., 0, 0, 0])

In [5]:
len(sum(np.where(ya==1)))

100

In [6]:
len(sum(np.where(ya==0)))

2608

`gen_structural_outlier`의 역할: Generating structural outliers

- 임의로 선택한 노드들이 fully connected 되어있을때 그 집단과 얼마나 많이 다른가??

In [7]:
data, ys = gen_structural_outlier(data, m=10, n=10)

In [8]:
ys

tensor([0, 0, 0,  ..., 0, 0, 0])

In [9]:
len(sum(np.where(ys==1)))

100

In [10]:
len(sum(np.where(ys==0)))

2608

위에서 찾은 이상치 간에 `torch.logical_or` 논리 or 생성

In [11]:
data.y = torch.logical_or(ys, ya).long()

In [12]:
data.y

tensor([0, 0, 0,  ..., 0, 0, 0])

In [13]:
len(sum(np.where(data.y==1)))

198

In [14]:
len(sum(np.where(data.y==0)))

2510

In [15]:
data = load_data('inj_cora')
data.y = data.y.bool()

In [16]:
data.y

tensor([False, False, False,  ..., False, False, False])

`self.bool` 로 참인지 거짓인지 가릴 수 있다.

- 여기서는 참이면 이상치인듯

'ANOMALOUS' 함수 사용, 일단 default로 다 지정함!

In [17]:
detector = ANOMALOUS(gamma=1.,
                     weight_decay=0.,
                     lr=0.004,
                     epoch=50,
                     gpu=-1,
                     contamination=0.1,
                     verbose=0)

In [18]:
detector.fit(data)

ANOMALOUS(contamination=0.1, epoch=50, gamma=1.0, gpu=None, lr=0.004,
          verbose=0, weight_decay=0.0)

```python
class ANOMALOUSBase(nn.Module):
    def __init__(self, w, r):
        super(ANOMALOUSBase, self).__init__()
        self.w = nn.Parameter(w)
        self.r = nn.Parameter(r)

    def forward(self, x):
        return x @ self.w @ x, self.r
```

In [19]:
detector.decision_function(data)

/home/csy/anaconda3/envs/temp_csy/lib/python3.8/site-packages/pygod/detector/anomalous.py:111: UserWarning: This detector is transductive only. Training from scratch with the input data.
  warnings.warn("This detector is transductive only. "


tensor([0.0293, 0.0284, 0.0283,  ..., 0.0304, 0.0295, 0.0287])

위에서 decision_function의 결과로 나오는 decision_score는 r의 제곱이며, 이 r은 model에서 나온 결과인데 이 model은 ANOMALOUSBase(w_init, r_init)의 결과이다.

이 r_init은 ANOMALOUS class 내에 있는 x, s, l, w_init, r_init = self.process_graph(data) 여기서 나온다.

`-` return되는 거는 순서대로 x, s, laplacian, w_init, r_init

`x`

In [20]:
detector.process_graph(data)[0]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [21]:
detector.process_graph(data)[0].shape

torch.Size([2708, 1433])

$X \in \mathbb{R}^{d \times n}$

2708 = `n` = the number of nodes

1433 = `d` = dimensiotnalattribute

`s`

In [22]:
detector.process_graph(data)[1]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 0.]])

In [23]:
detector.process_graph(data)[1].shape

torch.Size([2708, 2708])

$A \in \mathbb{R}^{n \times n}$

`laplacian`

In [24]:
detector.process_graph(data)[2]

tensor([[ 3.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  3., -1.,  ...,  0.,  0.,  0.],
        [ 0., -1.,  5.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  4., -1.],
        [ 0.,  0.,  0.,  ...,  0., -1.,  4.]])

In [25]:
detector.process_graph(data)[2].shape

torch.Size([2708, 2708])

`generated Laplacian`

$\tilde{R} L \tilde{R}^T$

`w_init`

In [26]:
detector.process_graph(data)[3]

tensor([[ 0.3903, -0.2553, -0.0085,  ...,  0.7819, -0.7382,  0.0120],
        [-1.2613, -0.4400, -0.7906,  ...,  0.1958,  0.4114,  0.1222],
        [ 0.6895,  0.5003, -1.1589,  ..., -2.6043, -2.4430,  0.1333],
        ...,
        [ 1.6715, -1.2318,  0.8302,  ..., -0.6394,  0.8524,  0.0301],
        [-0.6224, -2.6925,  2.3495,  ..., -0.9175, -1.1552, -0.2169],
        [-0.3202,  2.8018, -0.8151,  ...,  1.2948, -0.6545,  0.1930]])

In [27]:
detector.process_graph(data)[3].shape

torch.Size([1433, 2708])

`r_init`

In [28]:
detector.process_graph(data)[4]

tensor([[1.1096e-04, 1.9785e-04, 4.0764e-04,  ..., 1.8104e-05, 6.9121e-03,
         1.5390e-04],
        [1.8015e-05, 4.5550e-05, 1.5299e-04,  ..., 1.6462e-05, 8.0427e-03,
         4.3956e-05],
        [3.6342e-05, 1.1395e-04, 3.9942e-04,  ..., 3.5643e-05, 1.8391e-03,
         6.1595e-05],
        ...,
        [5.4851e-05, 1.2452e-04, 5.3171e-04,  ..., 1.6553e-05, 2.3374e-04,
         2.9871e-05],
        [2.3802e-04, 1.3623e-04, 5.8658e-04,  ..., 2.2358e-05, 1.6297e-04,
         4.2289e-04],
        [3.6244e-04, 1.9283e-04, 8.4447e-04,  ..., 3.0226e-05, 2.3002e-04,
         6.4480e-04]])

In [29]:
detector.process_graph(data)[4].shape

torch.Size([2708, 1433])